In [ ]:
#%pip install nltk
#%pip install --upgrade pip
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('stopwords')

In [ ]:
import pickle
with open("data.pkl", "rb") as f:
    questions_matrix, data, best_num_clusters, tfidf_vectorizer, model, history = pickle.load(f)

In [ ]:
data.head(3)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
import matplotlib.pyplot as plt

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocess_question(user_question):
    # Convertir la question en minuscules
    user_question = user_question.lower()

    # Supprimer la ponctuation
    user_question = re.sub(r'[^\w\s]', '', user_question)

    # Tokenization
    tokens = word_tokenize(user_question)

    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Reconstruire la question à partir des tokens lemmatisés
    processed_question = ' '.join(tokens)

    return processed_question

In [ ]:
user_question = preprocess_question(user_question) 

### Entrainement sans Modèle pour tester (Cette méthode est valide dans note cas d'étude,car le nombre de données n'est pas massive)

In [ ]:
user_question_vector = tfidf_vectorizer.transform([user_question])

print(user_question_vector.shape)
print(questions_matrix.shape)

cluster_similarities = cosine_similarity(user_question_vector, questions_matrix).flatten()

In [ ]:
cosine_similarities = cosine_similarity(user_question_vector, questions_matrix).flatten()
similarities_data = pd.DataFrame({
    'question_id': data['question_id'],
    'similarity': cosine_similarities,
    'cluster': data['cluster'],
    'answer_id': data['answer_id']
})

In [ ]:
similarities_data.head(4)

In [ ]:
print("Top 10 similar questions:")
print(similarities_data.sort_values(by='similarity', ascending=False).head(10))

In [ ]:
# Trouver les questions les plus similaires
most_similar_question = similarities_data.loc[similarities_data['similarity'].idxmax()]

# Cluster des questions les plus similaires
predicted_cluster = most_similar_question['cluster']
print(f"Predicted cluster for the new question: {predicted_cluster}")

In [ ]:
# Récupération des réponses du cluster prédit
cluster_responses = data[data['cluster'] == predicted_cluster]

# Calcul de la similarité entre la nouvelle question et les questions du cluster
cluster_cosine_similarities = cosine_similarity(user_question_vector, tfidf_vectorizer.transform(cluster_responses['body_x'])).flatten()

# Ajout des similarités au DataFrame du cluster
cluster_responses = cluster_responses.assign(similarity=cluster_cosine_similarities)

In [ ]:
# Récupérer les indices des réponses les plus similaires dans le cluster
most_similar_indices_in_cluster = cluster_responses['similarity'].nlargest(15).index

# Récupérer les réponses recommandées dans une liste
recommended_responses = cluster_responses.loc[most_similar_indices_in_cluster, 'body_y'].tolist()

# Supprimer les réponses dupliquées tout en préservant l'ordre
seen = set()
unique_recommended_responses = []
for response in recommended_responses:
    if response not in seen:
        unique_recommended_responses.append(response)
        seen.add(response)

# Afficher les réponses recommandées sous forme de liste
print("Les réponses recommandées sont:\n:")
for i, response in enumerate(unique_recommended_responses, start=1):
    print(f"{i}. {response}"+"\n*************************************************\n")

In [ ]:
# Convertir les similarités en DataFrame pour une meilleure visualisation
similarities_df = pd.DataFrame(cluster_similarities.T, index=data.index, columns=['similarity'])

# Ajouter les similarités au DataFrame original
data = data.join(similarities_df)

In [ ]:
# Identifier le cluster de la question la plus similaire
most_similar_question_idx = data['similarity'].idxmax()
predicted_cluster = data.loc[most_similar_question_idx, 'cluster']
print(predicted_cluster)

In [ ]:
# Sélectionner les réponses des questions du même cluster avec une similarité > 80%
recommended_responses = data[(data['cluster'] == predicted_cluster) & (data['similarity'] >= 0.5)][['answer_id', 'body_y']]
recommended_answers = pd.DataFrame(recommended_responses).drop_duplicates()

print("Les réponses recommandées sont:\n",recommended_answers['body_y'].head())

In [ ]:

df = data[['answer_id','view_count', 'cluster', 'is_answered', 'similarity']]
df['similarity'] = df['similarity'].apply(lambda x: f"{x*100:.2f}%")

# Convert the similarity values back to float for sorting
df['similarity_float'] = df['similarity'].str.rstrip('%').astype(float)

# Sort the DataFrame by the similarity column in descending order
df = df.sort_values(by='similarity_float', ascending=False)
df = df.drop(columns=['similarity_float'])

df = df.head(20)
df.head(15)

### Entrainement du modèle de classification / Prédiction

In [ ]:
# Prétraiter les questions
tfidf_vectorizer = TfidfVectorizer(max_features=20000)  
tfidf_matrix = tfidf_vectorizer.fit_transform(data['body_x'])

In [ ]:
# Créer et entraîner un modèle Keras
input_dim = tfidf_matrix.shape[1]

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=l2(0.001)))#256: le nombre de neurones, input_dim : nombre de features (vecteurs) que cette couche attend en entrée
#model.add(Dropout(0.3)) # 50%=0.5 : pourcentage de neurones à supprimer ou abondonner pour réduire l'overfitting.
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
#model.add(Dropout(0.1))
model.add(Dense(best_num_clusters, activation='softmax'))  

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraîner le modèle
X_train = tfidf_matrix.toarray()
y_train = data['cluster'].values

In [ ]:
history = model.fit(X_train, y_train, epochs=8, batch_size=64, validation_split=0.2)

In [ ]:
# Fonction pour recommander des réponses avec une similarité supérieure à 40%
def recommend_responses(new_question, model, df, tfidf_vectorizer, threshold=0.4, top_n=15):
    # Prétraiter la nouvelle question
    new_question_vector = tfidf_vectorizer.transform([new_question])

    # Prédire le cluster de la nouvelle question
    new_question_vector_array = new_question_vector.toarray()
    predicted_cluster = np.argmax(model.predict(new_question_vector_array), axis=1)[0]

    # Récupérer les questions et réponses du cluster prédit
    cluster_questions = df[df['cluster'] == predicted_cluster]

    # Calculer la similarité cosinus entre la nouvelle question et les questions du cluster
    cluster_question_vectors = tfidf_vectorizer.transform(cluster_questions['body_x'])
    similarities = cosine_similarity(new_question_vector, cluster_question_vectors).flatten()
    ########################
    #flatten():Retourne une copie d'un tableau donné de manière à ce qu'il soit réduit à une seule dimension.
    #Cela signifie que tous les éléments du tableau seront regroupés dans un seul tableau unidimensionnel
    #Exp: 1 2
    #     3 4
    # ==> 1 2 3 4
    ########################
    
    # Filtrer les réponses avec une similarité supérieure au seuil spécifié
    high_similarity_indices = [i for i, sim in enumerate(similarities) if sim > threshold]

    # Récupérer les réponses recommandées dans une liste
    recommended_responses = cluster_questions.iloc[high_similarity_indices]['body_y'].tolist()

    # Supprimer les réponses dupliquées tout en préservant l'ordre
    seen = set()
    unique_recommended_responses = []
    for response in recommended_responses:
        if response not in seen:
            unique_recommended_responses.append(response)
            seen.add(response)

    return unique_recommended_responses

In [ ]:
# Save the history
history_df = pd.DataFrame(history.history)
history_df = history_df.iloc[:15]

history_df.to_csv('history.csv', index=False)

In [ ]:
# Obtenir les réponses recommandées
recommended_responses = recommend_responses(user_question, model, data, tfidf_vectorizer)

# Afficher les réponses recommandées sous forme de liste
print("Recommended responses:")
for i, response in enumerate(recommended_responses, start=1):
    print(f"{i}. {response}")

In [ ]:
recommended_responses = pd.DataFrame(recommended_responses, columns=['Answers'])
recommended_responses.rename(columns={'0': 'Answers'}, inplace=True)
recommended_responses = recommended_responses.iloc[:15]

In [ ]:
recommended_responses.head(2)

In [ ]:
df.to_csv('data.csv',index=False)
recommended_responses.to_csv('recommendations.csv', index= False)

In [ ]:
# Tracer la courbe de la fonction de perte
plt.figure()
plt.plot(history.history['loss'], label='Entraînement')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Courbe de la fonction de perte')
plt.xlabel('Épochs')
plt.ylabel('Perte')
plt.savefig('loss_plot.png')
plt.legend()
plt.show()

In [ ]:
# Plot the accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Courbe de la précision du modèle')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('accuracy_plot.png')
plt.legend()
plt.close()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_pred = model.predict(X_train)
y_pred_clusters = np.argmax(y_pred, axis=1)

# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_train, y_pred_clusters)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)

# Afficher la matrice de confusion
plt.title('Matrice de Confusion')
plt.show()